### Load Dataset
This analysis comes from a case study created by *dqlab.id* site. The data used in this analysis comes from *dqlab.id* databases. The data contains information of sales performance on each office branch in many cities. But, for this analysis we only used the five largest province on the island of Java, Indonesia.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

# Load dataset
data_url = []
quarter_df = []
for idx in range(1,11,3):
    data_url.append("retail_data_from_{}_until_{}_reduce.csv".format(idx, idx+2))
for idx in range(4):
    df = pd.read_csv("https://storage.googleapis.com/dqlab-dataset/10%25_original_randomstate%3D42/" + data_url[idx])
    quarter_df.append(df)

In [2]:
# Get all the columns in each DataFrame
columns = {}
for idx, df in enumerate(quarter_df):
    columns["DataFrame {}".format(idx+1)] = df.columns.values

df_columns = pd.DataFrame(columns)
df_columns["same_name"] = [df_columns.iloc[row, :].unique().size==1 for row in range(df_columns.shape[0])]
print("COLUMNS ON EACH DATAFRAME:")
display(df_columns)

# Print out each DataFrame's shape
for idx in range(4):
    print("QUARTER {}: {}".format(idx + 1, quarter_df[idx].shape))

COLUMNS ON EACH DATAFRAME:


,DataFrame 1,DataFrame 2,DataFrame 3,DataFrame 4,same_name
0,order_id,order_id,order_id,order_id,True
1,order_date,order_date,order_date,order_date,True
2,customer_id,customer_id,customer_id,customer_id,True
3,city,city,city,city,True
4,province,province,province,province,True
5,product_id,product_id,product_id,product_id,True
6,brand,brand,brand,brand,True
7,quantity,quantity,quantity,quantity,True
8,item_price,item_price,item_price,item_price,True
9,total_price,total_price,total_price,total_price,True


QUARTER 1: (9489, 10)
QUARTER 2: (10123, 10)
QUARTER 3: (11061, 10)
QUARTER 4: (20183, 10)


### Concatenate DataFrame
Based on the output above, note that all of the columns in each DataFrame are the same. Hence we can combine all those DataFrame into one DataFrame. We can do this by using pandas' method called `.concat()`. See below:

In [3]:
# Concantenate DataFrame
retail_data = pd.concat([df for df in quarter_df])
retail_data.reset_index(drop=True, inplace=True)
display(retail_data.head(3))
display("SHAPE: {}".format(retail_data.shape))

# Print out DataFrame's info and some descriptive statistics
retail_data.info()
display(retail_data.describe())

,order_id,order_date,customer_id,city,province,product_id,brand,quantity,item_price,total_price
0,1612885,01-01-19,16293,Malang,Jawa Timur,P1301,BRAND_F,6,747000,4482000
1,1612387,01-01-19,17228,Bogor,Jawa Barat,P2086,BRAND_L,4,590000,2360000
2,1612903,01-01-19,16775,Surakarta,Jawa Tengah,P1656,BRAND_G,3,1325000,3975000


'SHAPE: (50856, 10)'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50856 entries, 0 to 50855
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   order_id     50856 non-null  object
 1   order_date   50856 non-null  object
 2   customer_id  50856 non-null  int64 
 3   city         50856 non-null  object
 4   province     50856 non-null  object
 5   product_id   50856 non-null  object
 6   brand        50856 non-null  object
 7   quantity     50856 non-null  int64 
 8   item_price   50856 non-null  int64 
 9   total_price  50856 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 3.9+ MB


,customer_id,quantity,item_price,total_price
count,50856.000000,50856.000000,5.085600e+04,5.085600e+04
mean,11516.434993,10.504680,1.345225e+06,7.020933e+06
std,6762.130438,41.966574,7.306866e+06,2.689227e+07
min,0.000000,1.000000,-8.910000e+05,-8.910000e+05
25%,12370.000000,1.000000,4.500000e+05,1.356000e+06
50%,14378.000000,4.000000,7.400000e+05,3.536000e+06
75%,16284.000000,11.000000,1.458000e+06,6.270000e+06
max,18287.000000,3114.000000,7.399120e+08,2.326158e+09


### Data Transforming
**Part 1:**
From the output above, notice that there is a negative minimum value in the **item_price** and **total_price** columns. This can affect our analysis, so we should drop it. 

In [4]:
# Slice only ambiguous data
below_zero = retail_data.loc[(retail_data['item_price']<0) | (retail_data['total_price']<0)]
display(below_zero)

# Drop ambiguous data
retail_data = retail_data[(retail_data['item_price']>0) & (retail_data['total_price']>0)]
print("NEW DATAFRAME'S SHAPE: {}".format(retail_data.shape))

# Print out data types of DataFrame's columns
print(retail_data.dtypes)

,order_id,order_date,customer_id,city,province,product_id,brand,quantity,item_price,total_price
4,1612915,01-01-19,0,unknown,unknown,P1230,BRAND_E,1,-891000,891000
96,1612927,01-01-19,0,unknown,unknown,P2736,BRAND_P,1,-891000,-891000
30694,1697299,01-10-19,17841,Jakarta Utara,DKI Jakarta,P2482,BRAND_P,1,520000,-520000


NEW DATAFRAME'S SHAPE: (50853, 10)
order_id       object
order_date     object
customer_id     int64
city           object
province       object
product_id     object
brand          object
quantity        int64
item_price      int64
total_price     int64
dtype: object


**Part 2:** If wee look at the data types of columns above, notice that **order_id** column has **object** data type. This is not possible if all **order_id**'s values are integers. So, there might be some *string* in that column. Let's examine this column. Also, because **order_date** column is still in **object** data type, we'll change it to **pandas' datetime** data type.

In [5]:
# Drop non-numeric/non-integer value on column 'order_id'
integer_ = []
for idx in retail_data['order_id']:
    if type(idx)==type(1234):
        integer_.append(True)
    else:
        integer_.append(idx.isdigit())

print("NON-NUMERIC ORDER_ID:")

display(retail_data.loc[~np.array(integer_), :])
retail_data = retail_data.loc[np.array(integer_), :]

# Change data type of 'order_id' column to integer
retail_data['order_id'] = retail_data['order_id'].astype('int64')
display(retail_data.head(3))
print("NEW DATAFRAME'S SHAPE: {}".format(retail_data.shape))

NON-NUMERIC ORDER_ID:


,order_id,order_date,customer_id,city,province,product_id,brand,quantity,item_price,total_price
19621,undefined,01-07-19,15150,Jakarta Pusat,DKI Jakarta,P3694,BRAND_T,12,159000,1908000


,order_id,order_date,customer_id,city,province,product_id,brand,quantity,item_price,total_price
0,1612885,01-01-19,16293,Malang,Jawa Timur,P1301,BRAND_F,6,747000,4482000
1,1612387,01-01-19,17228,Bogor,Jawa Barat,P2086,BRAND_L,4,590000,2360000
2,1612903,01-01-19,16775,Surakarta,Jawa Tengah,P1656,BRAND_G,3,1325000,3975000


NEW DATAFRAME'S SHAPE: (50852, 10)


In [6]:
# Change data type of 'order_date' column
retail_data['order_date'] = pd.to_datetime(retail_data['order_date'])

# Check all data types and print out descriptive statistics
print(retail_data.dtypes)
display(retail_data.describe())

order_id                int64
order_date     datetime64[ns]
customer_id             int64
city                   object
province               object
product_id             object
brand                  object
quantity                int64
item_price              int64
total_price             int64
dtype: object


,order_id,customer_id,quantity,item_price,total_price
count,5.085200e+04,50852.000000,50852.000000,5.085200e+04,5.085200e+04
mean,1.680619e+06,11516.692107,10.505211,1.345353e+06,7.021458e+06
std,3.867939e+04,6761.933325,41.968160,7.307137e+06,2.689326e+07
min,1.612372e+06,0.000000,1.000000,1.200000e+04,1.200000e+04
25%,1.646101e+06,12370.000000,1.000000,4.500000e+05,1.356000e+06
50%,1.682341e+06,14378.000000,4.000000,7.400000e+05,3.536000e+06
75%,1.714960e+06,16284.000000,11.000000,1.458000e+06,6.270000e+06
max,1.742998e+06,18287.000000,3114.000000,7.399120e+08,2.326158e+09


### Slice and Group By
Since our main objective in this notebook is to analyze only the top five largest province on the island of Java, we'll drop unnecessary columns and only preserve the data which are the five largest province on the island of Java. We achieved this by using pandas' method for DataFrame: `.groupby()`. See below:

In [7]:
# Select only the five largest province in the island of Java
provinces = ["DKI Jakarta", "Jawa Barat", "Jawa Tengah", "Jawa Timur", "Yogyakarta"]
retail_java = retail_data.loc[retail_data["province"].isin(provinces), :]
print("FIVE LARGEST PROVINCES ON JAVA ISLAND:")
display(retail_java.head())
print("PROVINCES: {}".format(retail_java['province'].unique()))
print("SHAPE: {}".format(retail_java.shape))

FIVE LARGEST PROVINCES ON JAVA ISLAND:


,order_id,order_date,customer_id,city,province,product_id,brand,quantity,item_price,total_price
0,1612885,2019-01-01,16293,Malang,Jawa Timur,P1301,BRAND_F,6,747000,4482000
1,1612387,2019-01-01,17228,Bogor,Jawa Barat,P2086,BRAND_L,4,590000,2360000
2,1612903,2019-01-01,16775,Surakarta,Jawa Tengah,P1656,BRAND_G,3,1325000,3975000
9,1612702,2019-01-01,18119,Yogyakarta,Yogyakarta,P3813,BRAND_U,1,450000,450000
14,1612951,2019-01-01,16125,Jakarta Barat,DKI Jakarta,P1902,BRAND_J,10,695000,6950000


PROVINCES: ['Jawa Timur' 'Jawa Barat' 'Jawa Tengah' 'Yogyakarta' 'DKI Jakarta']
SHAPE: (32735, 10)


In [8]:
# Export retail_java to .csv format
retail_java.to_csv('./retail_data_2019_java.csv')